## Import Python packages

In [1]:
import re
import numpy

from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow. keras.callbacks import ModelCheckpoint
import sys
import tensorflow as tf
tf.enable_eager_execution()
# tf.compat.v1.enable_eager_execution()

/opt/anaconda3/envs/debugger/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/debugger/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/debugger/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/debugger/lib/python3.7/site-packages/tensorf

## Load the data set

In [2]:
RAW_TEXT = open('lyrics-ds.txt', encoding = 'UTF-8').read()

## Create mapping of unique chars to integers

In [3]:
CHARS = sorted(list(set(RAW_TEXT)))
print('List of chars: \n', CHARS)

CHAR_TO_INT = dict((c, i) for i, c in enumerate(CHARS))
INT_TO_CHAR = dict((i, c) for i, c in enumerate(CHARS))

List of chars: 
 ['\n', ' ', '!', '"', '&', "'", ',', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


## Find out how many distinct characters our dataset has

In [4]:
TOTAL_CHARS = len(RAW_TEXT)
VOCAB_SIZE = len(CHARS)

print('Total Characters: ', TOTAL_CHARS) # 1.1M
print('Total Vocab: ', VOCAB_SIZE) # 46 distinct characters (much more than 26 in the alphabet)

Total Characters:  1103807
Total Vocab:  46


## Define hyperparameters for the learning algorithm

In [5]:
EPOCHS = 50     # the number times that the learning algorithm will work through the entire training dataset
BATCH_SIZE = 64 # the number of samples to work through before updating the internal model parameters
SEQ_LENGTH = 100

## Define the main functions of AI model lifecycle

In [6]:
def load_data():
    # prepare the data set of input to output pairs encoded as integers
    dataX = []
    dataY = []
    for i in range(0, TOTAL_CHARS - SEQ_LENGTH, 1):
        seq_in = RAW_TEXT[i:i + SEQ_LENGTH]
        seq_out = RAW_TEXT[i + SEQ_LENGTH]
        dataX.append([CHAR_TO_INT[char] for char in seq_in])
        dataY.append(CHAR_TO_INT[seq_out])
    n_patterns = len(dataX)

    # An example of a sequence length 3 in a dataset containing the text 'SAMPLE', the first 2 training patterns would be SAM -> P, AMP -> L
    #print ("Total Patterns: ", n_patterns) # a bit under 1.1M (TOTAL_CHARS - SEQ_LENGTH)

    # reshape X to be [samples, time steps, features]
    X = numpy.reshape(dataX, (n_patterns, SEQ_LENGTH, 1))

    # normalize
    X = X / float(VOCAB_SIZE)

    # one hot encode the output variable
    y = utils.to_categorical(dataY)

    return X,y

def create_model(X, y, layers):
    # define the LSTM model
    # Problem the model solves: single character classification problem with 46 classes (VOCAB_SIZE)
    # There is no test dataset. Model the entire training dataset to learn the probability of each character in a sequence
    print("Creating model...")
    model = Sequential()
    
    for n in range(layers-2):
        # add hidden LSTM layer with 256 memory units
        model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True)) 
        model.add(Dropout(0.2))
    model.add(LSTM(256))
    model.add(Dropout(0.2))
    model.add(Dense(y.shape[1], activation='softmax')) # outputs a probability prediction for each of the 46 characters between 0 and 1
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

def train(X, y, checkpoint_file=None):
    model = create_model(X, y)

    # use model checkpointing for optimization (too slow to train)
    # record all of the network weights to file each time
    # an improvement in loss is observed at the end of the epoch
    # then the best set of weights (lowest loss) to instantiate the generative model in the next section

    # define the checkpoint
    # filepath = 'lstm-4-layers-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5'
    filepath = 'weights-improvement-{epoch:02d}-{loss:.4f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]

    if checkpoint_file:
        print("Loading checkpoint: " + checkpoint_file)
        model.load_weights(checkpoint_file)

    # Fit model to the data (for now use 50 epochs and a medium batch size of 64 patterns)
    model.fit(X, y, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks_list, shuffle=True)

def generate_lyric(X, y, weights_file):
    layers = int(weights_file[0])
    model = create_model(X, y, layers)

    # load the network weights
    model.load_weights(weights_file)
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    n_patterns = TOTAL_CHARS - SEQ_LENGTH
    start = numpy.random.randint(0, n_patterns-1)
    pattern = [CHAR_TO_INT[char] for char in RAW_TEXT[start:start+SEQ_LENGTH]]
    output = [INT_TO_CHAR[value] for value in pattern]

#     print('Seed:')
#     print(''.join(output))

    # generate characters
    for i in range(1000):
        X = numpy.reshape(pattern, (1, len(pattern), 1))
        X = X / float(VOCAB_SIZE)
        prediction = model.predict(X, verbose=0)
        # index = numpy.argmax(prediction)
        # index = tf.random.categorical(prediction, 1)[-1,0].numpy()
        index = numpy.random.choice(len(prediction[0]), p=prediction[0])
        result = INT_TO_CHAR[index]
        output.append(result)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return ''.join(output)

## Load the data

In [7]:
x, y = load_data()

In [8]:
### We are skipping data training for now to use checkpoints

## Generate lyric 1
### Using a 3-lstem-layer model trained in 50 epochs

In [9]:
print('\nOUTPUT:\n', generate_lyric(x, y, "3-layers-weights-improvement-50-1.4956.hdf5"))

Creating model...

OUTPUT:
 e message
and the lunatic fears
he is no more
speak to me
it all would be easier
while i'm passing blood
eornaga i uidy our life
far within the facts it's night of touer
id you fuer lielt 'caat. fireer is laie tired
and it shall
be ovr ceaays that lies in micning of all closds
wouldst the endless warted weaross orr
has done to astitl drysted our wing
fqom the vipter tuonid vincre of laws
iu's wearing dust cowr, tom pain
i ac monn agpny
uhere' dom't give up this hades
but whine ttrongernidd is meft
in elbctiwarions
of ny sidd
sandlisy yill hase the baai
teostality pows
loog anrwer to sav jt yith variols troa
hone of bloodlagd percprelp
the faredn nf monn' fade and tpptire
do mook on me weak away
far beyond it feel
in a buedc, earaasy, a worl heals say hreatet spoebns
and thou all tavs tires tife so paressinn
the mine's aro oh halr
bedined it a sunf uo to tnm' a land of wearing mnuier
but no winter fights befins the stnesw
tatte my asms
to acr of a world blis 

## Generate lyric 2
### Using a 5-lstem-layer model trained in 20 epochs

In [10]:
print('\nOUTPUT:\n------------------\n', generate_lyric(x, y, "5-layers-weights-improvement-20-1.5581.hdf5"))

Creating model...

OUTPUT:
------------------
  the engines
remove all the wheel blocks there's no time to waste
gathering speed as we head down there's now i'll hear
nor be the
whispered words
we will be reftsed tonight
has all ever mose is yiere smoky
you will cone to athve
the price i see me
the hravekace is put the masser fortalen
forever reborn carkness will say
but they remossed
by this peace to see?
the darkness suarling now nochess
wasted in the mast behende, behind why i'll sing on
sppn when you befores that carknning once
dark is savisued in gatition
the wind of solendornat cruel bgainstuine
cruelfing lives fron the vay and afhiaai
dead and nowering, spirits
fly on it in her shothh
all
has a winter collc we have out cack
dividen smeepes ruent from the lneelsaah the wokn
so pain, drawy running
like now for gormt
fuurom cy the burning one
the days i coundn't gone
strong to see it all i'm now answers on
i pus her all prichory ruegtction out my stinps
a faces cedale
my sins now a

Note that more lstm layers means deeper and more accurate learning (also more time processing the training).
Not only this model is learns how to write lyrics, just like humans it also needs to learn how to make sense of letter and come up with meaningful words :)